In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import sys
import time
import pathlib
import datetime
import locale
from os import path
locale.setlocale(locale.LC_ALL, 'Turkish')

url = "https://www.bddk.org.tr/BultenHaftalik/"
driver = webdriver.Chrome('chromedriver')
time.sleep(3)
driver.get(url)
time.sleep(3)

tar = '-'.join(map(str,[driver.find_element("xpath",'//*[@id="Yil"]/option[1]').text] + driver.find_element("xpath",'//*[@id="Donem"]/option[1]').text.split(' ')[0].split('/')))
tarih = datetime.datetime.strptime(tar,'%Y-%B-%d')

def GetWeeklyBDDK():    
    table = []
    tarafDict = {10001:500, 10002:501, 10008:502, 10009:503, 10010:504, 10003:505, 10004:506, 10005:507, 10006:508,10007:511}
    tabloList = [280,281,282,283,284,285]
    
    tlypList = ["T","Y","P"]
    for t in tarafDict.keys():
        driver.find_element("xpath",'//*[@id="tarafListesiItem-'+str(t)+'"]').click()
        time.sleep(3)
        taraf = driver.find_element("xpath",'//*[@id="tarafListesiItem-'+str(t)+'"]').text
        kurumKod = tarafDict[t]
        metrik_no = 9001
        for k in tabloList:
            driver.find_element("xpath",'//*[@id="tabloListesiItem-'+str(k)+'"]').click()
            time.sleep(3)
            kalem = driver.find_element("xpath",'//*[@id="tabloListesiItem-'+str(k)+'"]').text
            for i in range(1,len(driver.find_elements("xpath",'//*[@id="Tablo"]/tbody/tr')) + 1):
                if (driver.find_element("xpath",'//*[@id="Tablo"]/tbody/tr['+str(i)+']/td[2]').text != ''):
                    count = 0
                    metrik_no += 1
                    for j in range(3,len(driver.find_elements("xpath",'//*[@id="Tablo"]/tbody/tr[1]/td')) + 1):
                        table.append([tarih,
                                      taraf,
                                      kurumKod,
                                      kalem,
                                      tlypList[count],
                                      metrik_no,
                                      driver.find_element("xpath",'//*[@id="Tablo"]/tbody/tr['+str(i)+']/td[2]').text,
                                      driver.find_element("xpath",'//*[@id="Tablo"]/tbody/tr['+str(i)+']/td['+str(j)+']').text])
                        count += 1
    driver.close()
    
    df = pd.DataFrame(table,columns = ['END_DT','TARAF','BNK_KRM_KOD','KALEM','TL_YP_KOD','METRIK_NO','METRIK','METRIK_DEGER'])
    
    df.to_excel("Haftalık_Sektor.xlsx")
    f = open("Haftalık_Sektor.txt", "w")
    f.write(tar)
    f.close()

if not(path.exists('Haftalık_Sektor.txt')):
    GetWeeklyBDDK()
    sys.exit()
else:   
    txtRead = open("Haftalık_Sektor.txt","r")
    txtTar = txtRead.read()
    txtRead.close()
    
    if tar == txtTar:
        driver.close()
        sys.exit()
    else:
        GetWeeklyBDDK()
        sys.exit()

SystemExit: 

C:\Users\Onur Parapan\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
